# DeFiLlama Price Puller Example

This notebook demonstrates how to use the new DeFiLlama price puller functionality.

The DeFiLlama price puller supports:
- Current prices
- Historical prices with flexible date ranges
- First recorded prices
- Both CoinGecko slugs and chain:address format
- Custom token lists from files
- Gas tokens from chain metadata


In [4]:
import sys
sys.path.append('../../..')

from src.op_analytics.datasources.defillama.tokenprice.execute import (
    execute_pull_current,
    execute_pull_historical,
    execute_pull_first_prices,
)
from src.op_analytics.datasources.defillama.tokenprice.price_data import DefiLlamaTokenPrices
from op_analytics.datapipeline.chains.tokens import get_token_ids_from_metadata
from op_analytics.coreutils.request import new_session
from datetime import datetime, timedelta
import polars as pl

# Optional: Set up logging
import logging
logging.basicConfig(level=logging.INFO)


## 1. Get Token IDs from Chain Metadata

This will get both CoinGecko slugs and gas tokens from the chain metadata:


In [2]:
# Get token IDs from chain metadata
token_ids = get_token_ids_from_metadata()
print(f"Found {len(token_ids)} token IDs from chain metadata")
print("First 10 token IDs:", token_ids[:10])


2025-07-09 15:33:46 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=43292
2025-07-09 15:33:46 [debug    ] loaded vault: 28 items         filename=vault.py lineno=79 process=43292
2025-07-09 15:33:46 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=43292
2025-07-09 15:33:48 [info     ] Loaded OP chains metadata from /Users/michaelsilberling/Documents/GitHub/op-analytics/op_chains_tracking/inputs/chain_metadata_raw.csv filename=load.py lineno=87 process=43292
2025-07-09 15:33:48 [info     ] [REPO vs. GSHEETS] ERROR: Chain Metadata is different filename=load.py lineno=99 process=43292
2025-07-09 15:33:48 [info     ] found_token_ids                coingecko_count=12 count=27 filename=tokens.py gas_token_count=15 lineno=58 process=43292
Found 27 token IDs from chain metadata
First 10 token IDs: ['automata', 'SYS', 'RSS3', 'GSWIFT', 'ETH', 'celo', 'mantle', 'frxETH', 'rss3', 'hashkey-ecopoints']


## 2. Direct API Usage (Without Writing to GCS)

Use the price data class directly for analysis:


In [3]:
session = new_session()

# Get current prices for multiple tokens
tokens = ["bitcoin", "ethereum", "ethereum:0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"]
price_data = DefiLlamaTokenPrices.fetch_prices_current(tokens, session=session)

print(f"Fetched {len(price_data)} current prices")
print(price_data.df.head())

# Get historical prices for the last 7 days 
# This uses start timestamp + span=days to get exactly 7 daily data points from the start date
price_data = DefiLlamaTokenPrices.fetch_prices_by_days(
    token_ids=tokens,
    days=7,
    session=session
)

print(f"\\nFetched {len(price_data)} historical prices")
print(price_data.df.head())

# Sort by token_id and date to see the progression
print("\\nHistorical prices sorted by token and date:")
historical_sorted = price_data.df.sort(["token_id", "dt"])
print(historical_sorted)

# Basic analysis
print("\\nPrice summary:")
print(price_data.df.group_by("token_id").agg([
    pl.col("price_usd").min().alias("min_price"),
    pl.col("price_usd").max().alias("max_price"),
    pl.col("price_usd").mean().alias("avg_price"),
    pl.col("dt").count().alias("data_points"),
    pl.col("dt").min().alias("min_dt"),
    pl.col("dt").max().alias("max_dt"),
]))


2025-07-09 15:33:49 [info     ] fetched_current_prices         filename=price_data.py lineno=150 process=43292 token_count=3
Fetched 3 current prices
shape: (3, 4)
┌─────────────────────────────────┬────────────┬───────────┬────────────────────────────┐
│ token_id                        ┆ dt         ┆ price_usd ┆ last_updated               │
│ ---                             ┆ ---        ┆ ---       ┆ ---                        │
│ str                             ┆ str        ┆ f64       ┆ str                        │
╞═════════════════════════════════╪════════════╪═══════════╪════════════════════════════╡
│ bitcoin                         ┆ 2025-07-09 ┆ 109718.0  ┆ 2025-07-09T15:33:49.638084 │
│ ethereum                        ┆ 2025-07-09 ┆ 2723.41   ┆ 2025-07-09T15:33:49.638084 │
│ ethereum:0xa0b86991c6218b36c1d… ┆ 2025-07-09 ┆ 0.999902  ┆ 2025-07-09T15:33:49.638084 │
└─────────────────────────────────┴────────────┴───────────┴────────────────────────────┘
2025-07-09 15:33:51 [info 

In [ ]:
# Overall run
from op_analytics.coreutils.partitioned.dailydatawrite import write_to_prod

# Path to your config file
extra_token_ids_file = "../../../src/op_analytics/datasources/coingecko/config/extra_token_ids.txt"
with write_to_prod():
    result = execute_pull_historical(days=365)

2025-07-09 15:38:27 [info     ] found_token_ids                coingecko_count=12 count=27 filename=tokens.py gas_token_count=15 lineno=58 process=43292
2025-07-09 15:38:27 [info     ] final_token_ids                count=27 filename=tokens.py lineno=154 process=43292
2025-07-09 15:38:27 [info     ] historical_price_pull_start    filename=execute.py lineno=136 process=43292 token_count=27
2025-07-09 15:38:27 [info     ] fetching_historical_prices     filename=execute.py lineno=144 process=43292 token_count=27
2025-07-09 15:40:58 [error    ] retrying exception HTTPSConnectionPool(host='pro-api.llama.fi', port=443): Max retries exceeded with url: /slPjmq113xSROlwRStlysKOhP0coMlgQkcPd0lgLMV3sL316g8l8CQ/coins/chart/coingecko:automata,coingecko:SYS,coingecko:RSS3,coingecko:GSWIFT,coingecko:ETH,coingecko:celo,coingecko:mantle,coingecko:frxETH,coingecko:rss3,coingecko:hashkey-ecopoints,coingecko:frax-ether,coingecko:BNB,coingecko:ATA,coingecko:unknown,coingecko:syscoin,coingecko:BNRY,coingeck

## Summary

The DeFiLlama price puller provides:

1. **Three main execution modes**: `current`, `historical`, and `first`
2. **Flexible token identification**: CoinGecko slugs, chain:address format, custom files
3. **Automatic chain metadata integration**: Pulls gas tokens and CoinGecko IDs from your chain metadata
4. **Full DeFiLlama API parameter support**: timestamps, periods, spans, search width
5. **GCS integration**: Writes data to your existing data pipeline
6. **Direct API access**: Use the price data classes for analysis without writing to GCS

### Command Line Usage:

```bash
# Current prices
python -m src.op_analytics.datasources.defillama.tokenprice.execute --mode current

# Historical prices (last 30 days)
python -m src.op_analytics.datasources.defillama.tokenprice.execute --mode historical --days 30

# Custom historical with timestamps
python -m src.op_analytics.datasources.defillama.tokenprice.execute --mode historical --start-timestamp 1704067200 --period 1h

# Single token
python -m src.op_analytics.datasources.defillama.tokenprice.execute --mode current --token-id bitcoin

# With custom token file
python -m src.op_analytics.datasources.defillama.tokenprice.execute --mode current --extra-token-ids-file /path/to/tokens.txt
```

### Key Features:

- **Support for both formats**: CoinGecko slugs (e.g., "bitcoin") and chain:address format (e.g., "ethereum:0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48")
- **Automatic gas token detection**: Pulls gas tokens from your chain metadata
- **Flexible time ranges**: Use days, timestamps, or first recorded prices
- **Custom token lists**: Load tokens from CSV or TXT files
- **Skip existing partitions**: Efficient backfilling with `--skip-existing-partitions`
